In [28]:
import xarray as xr
import numpy as np
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
from pyproj import Geod
import matplotlib.gridspec as gridspec
import sys
try:
    from sklearn.metrics import r2_score, root_mean_squared_error
except:
    from sklearn.metrics import r2_score, mean_squared_error

def calc_metrics(obs, mod):
    cleaned = obs.dropna()
    mod = mod.loc[mod.index.isin(cleaned.index)]
    cleaned = cleaned.loc[cleaned.index.isin(mod.index)]
    try:
        rmse = root_mean_squared_error(cleaned, mod)
    except:
        rmse = mean_squared_error(cleaned,mod, squared=False)
    r2 = r2_score(cleaned, mod)
    print(rmse,r2)
    
    mae = np.abs(mod-cleaned)
    mae = np.nanmean(mae)
    return (rmse,r2,mae)

plt.rcParams.update({'font.size': 24})

In [ ]:
## TASK FOR TODAY: Take existing scripts and plot the ENS MEAN and Highlight the specific simulations in there! 
## Then extend Hypsometry plot to also show lapse rate of SNOWFALL and REFREEZE for example
#Load and prepare WGMS data
condition = "bestfiles" #bestfiles or bestsr50 lowsnow_bestfiles

if 'win' in sys.platform:
    path = f"E:/OneDrive/PhD/PhD/Data/Hintereisferner/Output/hypso/{condition}/"
    enspath = f"E:/OneDrive/PhD/PhD/Data/Hintereisferner/Output/hypso/{condition}/"
    wgms_path = "E:/OneDrive/PhD/PhD/Data/DOI-WGMS-FoG-2022-09/data/"
else:
    path = f"/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Output/hypso/{condition}/"
    enspath = f"/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Output/hypso/{condition}/"
    wgms_path = "/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/DOI-WGMS-FoG-2022-09/data/"

wgms = pd.read_csv(wgms_path+"mass_balance.csv")
wgms = wgms.loc[(wgms['NAME'] == "HINTEREIS F.") & (wgms['YEAR'] >= 2002) & (wgms['YEAR'] <= 2009)] ## EXCLUDE 2001 - does not feature same elevation bands as other datasets
print(wgms.NAME.iloc[0], np.unique(wgms.WGMS_ID))
wgms.drop(['POLITICAL_UNIT', 'NAME','REMARKS'], axis=1, inplace=True)
mb = wgms.groupby('LOWER_BOUND').mean()
mb.index = mb.index+25
mb

In [ ]:
### Check WGMS area over time
df_filtered = wgms[(wgms['LOWER_BOUND'] != 9999) & (wgms['UPPER_BOUND'] != 9999)]
df_filtered['ELEVATION_BAND'] = df_filtered['LOWER_BOUND'].astype(str) + '-' + df_filtered['UPPER_BOUND'].astype(str)
area_pivot = df_filtered.pivot_table(index='YEAR', columns='ELEVATION_BAND', values='AREA', aggfunc='sum')
area_change = area_pivot.subtract(area_pivot.iloc[0], axis=1)
area_change.plot(figsize=(12, 6), colormap='plasma')
plt.title('Change in Glacier Area by Elevation Band (Relative to First Year)')
plt.xlabel('Year')
plt.ylabel('Area Change (km²)')
plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
plt.legend(title='Elevation Band', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

In [ ]:
mb_2004 = wgms.loc[wgms['YEAR'] == 2004].groupby('LOWER_BOUND').mean()
mb_2004.index = mb_2004.index+25
mb_2004

In [ ]:
### Calculate x and y size of grid - inspired by chatgpt... fix?!
if 'win' in sys.platform:
    test = xr.open_dataset(r"E:\OneDrive\PhD\PhD\Data\Hintereisferner\Static\HEF_static_30m_new.nc")
else:
    test = xr.open_dataset("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Static/HEF_static_30m_new.nc")
lat = test['lat'].values
lon = test['lon'].values

# Initialize the geod object (WGS84 ellipsoid)
geod = Geod(ellps="WGS84")

# Calculate the size in the y direction (latitude spacing)
lat_mid = (lat[:-1] + lat[1:]) / 2  # Midpoint latitude for more accurate distance
y_size = []
for i in range(len(lat) - 1):
    _, _, distance = geod.inv(lon[0], lat[i], lon[0], lat[i + 1])  # Distance between latitudes
    y_size.append(distance)

# Calculate the size in the x direction (longitude spacing)
x_size = []
for j in range(len(lon) - 1):
    _, _, distance = geod.inv(lon[j], lat_mid[0], lon[j + 1], lat_mid[0])  # Distance between longitudes
    x_size.append(distance)

# Convert results to arrays for grid size
x_size = np.array(x_size)
y_size = np.array(y_size)

# Add these to the dataset if needed
test['x_size'] = (('longitude',), x_size)  # Distance in x direction
test['y_size'] = (('latitude',), y_size)   # Distance in y direction

# Save or inspect the results
print("X-direction sizes (m):", x_size.mean())
print("Y-direction sizes (m):", y_size.mean())

In [ ]:
#get area per elevation bin
if 'win' in sys.platform:
    ds = xr.open_dataset("E:/OneDrive/PhD/PhD/Data/Hintereisferner/COSIPY/MiscTests/LHS/EB_Output/HEF_COSMO_1D20m_1999_2010_HORAYZON_IntpPRES_MCMC-ensemble_19990101-20091231_RRR-0.7409_0.8852_0.2279_0.6388_13.82_1.0188_0.24_1.7216_4.0_0.0026_num274.nc")
else:
    ds = xr.open_dataset("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/COSIPY/MiscTests/LHS/EB_Output/HEF_COSMO_1D20m_1999_2010_HORAYZON_IntpPRES_MCMC-ensemble_19990101-20091231_RRR-0.7409_0.8852_0.2279_0.6388_13.82_1.0188_0.24_1.7216_4.0_0.0026_num274.nc")
bins = np.arange(2400,3700+50,50) #from WGMS bins -> first band disregard, bc. min elev 2440
labels= bins[:-1]+25
area_ds = ds['N_Points'].groupby_bins(ds.HGT, bins, labels=labels, include_lowest=True).sum(skipna=True,min_count=1)
area_per_bin = area_ds.values * (21.281370201526517/1000) * (30.879719652645804/1000) #roughly
np.nansum(area_per_bin)

In [ ]:
## Load Ensemble
df_ensemble_holder = pd.DataFrame(index= labels)
for fp in pathlib.Path(enspath).glob('hypsometry_cspy_HEF_COSMO_1D20m_1999_2010_HORAYZON_IntpPRES*.csv'):
    #print(fp)
    df = pd.read_csv(fp, index_col=0)
    if condition == "lowsnow_bestfiles":
        raw_fp = str(fp.stem).split('hypsometry_cspy_HEF_COSMO_1D20m_1999_2010_HORAYZON_IntpPRES_19990101-20091231_RRR-')[-1]
    else:
        raw_fp = str(fp.stem).split('hypsometry_cspy_HEF_COSMO_1D20m_1999_2010_HORAYZON_IntpPRES_MCMC-ensemble_19990101-20091231_RRR-')[-1]
    #print(raw_fp)
    rrr_factor = float(raw_fp.split('_')[0])
    alb_snow = float(raw_fp.split('_')[1])
    alb_ice = float(raw_fp.split('_')[2])
    alb_firn = float(raw_fp.split('_')[3])
    alb_aging = float(raw_fp.split('_')[4])
    alb_depth = float(raw_fp.split('_')[5])
    roughness_fresh_snow = float(raw_fp.split('_')[6])
    roughness_ice = float(raw_fp.split('_')[7])
    roughness_firn = float(raw_fp.split('_')[8])
    aging_factor_roughness = float(raw_fp.split('_')[9])
    
    name = f"{rrr_factor}_{alb_snow}_{alb_ice}_{alb_firn}_{alb_aging}_{alb_depth}_{roughness_ice}"
    print(name)
    df_ensemble_holder[name] = df['MB']
df_ensemble_holder

ens_df = pd.DataFrame(index=labels)
ens_df['ENS_MEAN'] = df_ensemble_holder.mean(axis=1)
ens_df['ENS_MEDIAN'] = df_ensemble_holder.median(axis=1)
ens_df['ENS_STD'] = df_ensemble_holder.std(axis=1)
ens_df['ENS_MIN'] = df_ensemble_holder.min(axis=1)
ens_df['ENS_MAX'] = df_ensemble_holder.max(axis=1)
ens_df['CI_lower'] = df_ensemble_holder.quantile(0.025, axis=1)
ens_df['CI_upper'] = df_ensemble_holder.quantile(0.975, axis=1)
ens_df = ens_df.replace(0.0,np.nan)
ens_df

In [ ]:
## Load Ensemble

df_ensemble_holder_2004 = pd.DataFrame(index= labels)
#hypsometry_cspy_YEAR2004_HEF_COSMO_1D20m_1999_2010_HORAYZON_LHS_19990101-20091231_RRR-0.6747_0.8933_0.38_0.5912_25.9099_3.7313_0.24_1.7_4.0_0.0026_num2.csv
for fp in pathlib.Path(enspath).glob('hypsometry_cspy_YEAR2004_HEF_COSMO_1D20m_1999_2010_HORAYZON_IntpPRES*.csv'):
    print(fp)
    df = pd.read_csv(fp, index_col=0)
    if condition == "lowsnow_bestfiles":
        raw_fp = str(fp.stem).split('hypsometry_cspy_HEF_HEF_COSMO_1D20m_1999_2010_HORAYZON_LHSnoRRR_19990101-20091231_RRR-')[-1]
    else:
        raw_fp = str(fp.stem).split('hypsometry_cspy_YEAR2004_HEF_COSMO_1D20m_1999_2010_HORAYZON_IntpPRES_MCMC-ensemble_19990101-20091231_RRR-')[-1]
    rrr_factor = float(raw_fp.split('_')[0])
    alb_snow = float(raw_fp.split('_')[1])
    alb_ice = float(raw_fp.split('_')[2])
    alb_firn = float(raw_fp.split('_')[3])
    alb_aging = float(raw_fp.split('_')[4])
    alb_depth = float(raw_fp.split('_')[5])
    roughness_fresh_snow = float(raw_fp.split('_')[6])
    roughness_ice = float(raw_fp.split('_')[7])
    roughness_firn = float(raw_fp.split('_')[8])
    aging_factor_roughness = float(raw_fp.split('_')[9])
    
    name = f"{rrr_factor}_{alb_snow}_{alb_ice}_{alb_firn}_{alb_aging}_{alb_depth}_{roughness_ice}"
    print(name)
    df_ensemble_holder_2004[name] = df['MB']
df_ensemble_holder_2004

ens_df_2004 = pd.DataFrame(index=labels)
ens_df_2004['ENS_MEAN'] = df_ensemble_holder_2004.mean(axis=1)
ens_df_2004['ENS_MEDIAN'] = df_ensemble_holder_2004.mean(axis=1)
ens_df_2004['ENS_STD'] = df_ensemble_holder_2004.std(axis=1)
ens_df_2004['ENS_MIN'] = df_ensemble_holder_2004.min(axis=1)
ens_df_2004['ENS_MAX'] = df_ensemble_holder_2004.max(axis=1)
ens_df_2004['CI_lower'] = df_ensemble_holder_2004.quantile(0.025, axis=1)
ens_df_2004['CI_upper'] = df_ensemble_holder_2004.quantile(0.975, axis=1)
ens_df_2004 = ens_df_2004.replace(0.0,np.nan)
ens_df_2004

In [36]:
## Load MB hypsometry for 2003/04 from WGMS ? 


In [37]:
total_mb = df_ensemble_holder.sum(axis=0)

most_negative_run = total_mb.idxmin()  # Column with the smallest total MB
most_positive_run = total_mb.idxmax()  # Column with the largest total MB

ens_df["Physical_Min"] = df_ensemble_holder[most_negative_run]
ens_df["Physical_Max"] = df_ensemble_holder[most_positive_run]



In [ ]:
ens_df

In [39]:
#df_holder_sf = pd.DataFrame(index= labels)
#for fp in pathlib.Path(path).glob('hypsometry_cspy_HEF*.csv'):
#    print(fp)
#    df = pd.read_csv(fp, index_col=0)
#    key = str(fp.stem).split('_')[-1].split('.csv')[0]
#    print(key)
#    df_holder_sf[key] = df['SNOWFALL']
#df_holder_sf

In [40]:
#df_holder_t2 = pd.DataFrame(index= labels)
#for fp in pathlib.Path(path).glob('hypsometry_cspy_HEF*.csv'):
#    print(fp)
#    df = pd.read_csv(fp, index_col=0)
#    key = str(fp.stem).split('_')[-1].split('.csv')[0]
#    print(key)
#    df_holder_t2[key] = df['T2']
#df_holder_t2

In [41]:
#df_holder_rrr = pd.DataFrame(index= labels)
#for fp in pathlib.Path(path).glob('hypsometry_cspy_HEF*.csv'):
#    print(fp)
#    df = pd.read_csv(fp, index_col=0)
#    key = str(fp.stem).split('_')[-1].split('.csv')[0]
#    print(key)
#    df_holder_rrr[key] = df['RRR']
#df_holder_rrr

In [ ]:
## Load annual mb data
##seasonal mb from HEF
if 'win' in sys.platform:
    wgms_mb = pd.read_csv(r"e:\OneDrive\PhD\PhD\Data\DOI-WGMS-FoG-2022-09\data\mass_balance.csv")
else:
    wgms_mb = pd.read_csv("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/DOI-WGMS-FoG-2022-09/data/mass_balance.csv")
wgms_mb = wgms_mb.loc[wgms_mb['NAME'] == "HINTEREIS F."]
wgms_mb = wgms_mb.loc[(wgms_mb['YEAR'] >= 2000) & (wgms_mb['YEAR'] < 2011)]
wgms_mb = wgms_mb.loc[wgms_mb['LOWER_BOUND'] == 9999]
wgms_mb.drop(['POLITICAL_UNIT','NAME'], axis=1, inplace=True)
wgms_mb['YEAR'] = pd.to_datetime(wgms_mb['YEAR'], format='%Y')
wgms_mb

In [ ]:
## Load annual MB from simulations
if 'win' in sys.platform:
    mb_sims = pd.read_csv("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Output/current/bestfiles/AvgSpatMB_ens.csv", index_col=0, parse_dates=True)
else:
    mb_sims = pd.read_csv("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Output/current/bestfiles/AvgSpatMB_ens.csv", index_col=0, parse_dates=True)
year_vals = np.arange(2002,2009+1,1) #2002
mb_holder = np.zeros(shape=(len(year_vals),mb_sims.shape[1]))
#time_slices_wy= [('2000-10-01','2001-09-30'),('2001-10-01','2002-09-30'),('2002-10-01','2003-09-30'),('2003-10-01','2004-09-30'),
time_slices_wy= [('2001-10-01','2002-09-30'),('2002-10-01','2003-09-30'),('2003-10-01','2004-09-30'),
                 ('2004-10-01','2005-09-30'),('2005-10-01','2006-09-30'),('2006-10-01','2007-09-30'),('2007-10-01','2008-09-30'),('2008-10-01','2009-09-30')]

i=0
for tslice in time_slices_wy:
                print(tslice)
                sub = mb_sims.loc[tslice[0]:tslice[1]].sum()
                mb_holder[i,:] = sub.values
                i+=1


In [ ]:
column_names = mb_sims.columns  # Extract column names

# Create the new dataframe
df_new = pd.DataFrame(data=mb_holder, index=pd.to_datetime(year_vals, format="%Y"), columns=column_names)
df_new = df_new * 1000

median_mb = df_new.median(axis=1)
ci_lower_mb = df_new.quantile(0.025, axis=1)
ci_upper_mb = df_new.quantile(0.975, axis=1)
median_mb

In [ ]:
ci_lower_mb

In [ ]:
ci_upper_mb

In [ ]:
fig, ax = plt.subplots(1,1, dpi=150)
ax.plot(wgms_mb['YEAR'], wgms_mb['ANNUAL_BALANCE'], color="black", label="MB")
ax.fill_between(x=wgms_mb['YEAR'], y1=wgms_mb['ANNUAL_BALANCE']-wgms_mb['ANNUAL_BALANCE_UNC'], y2=wgms_mb['ANNUAL_BALANCE']+wgms_mb['ANNUAL_BALANCE_UNC'])

ax.plot(median_mb.index, median_mb.values, label="ENS MEAN", color="red")
ax.fill_between(x=median_mb.index, y1=ci_lower_mb, y2=ci_upper_mb, color="brown", alpha=0.5, label = "$\pm$ Std. deviation")
for column in df_new.columns:
    ax.plot(df_new.index, df_new[column], alpha=0.2, linewidth=1., zorder=6, color="brown")
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
ax.legend()

In [ ]:
mb = mb[['ANNUAL_BALANCE','ANNUAL_BALANCE_UNC','AREA']] #mb[mb.index.isin(ens_df.index)]
mb['ANNUAL_BALANCE'] = mb['ANNUAL_BALANCE']/1000
mb.drop(mb.tail(1).index,inplace=True) # drop last n rows

mb_2004 = mb_2004[['ANNUAL_BALANCE','ANNUAL_BALANCE_UNC','AREA']] #mb[mb.index.isin(ens_df.index)]
mb_2004['ANNUAL_BALANCE'] = mb_2004['ANNUAL_BALANCE']/1000
mb_2004.drop(mb.tail(1).index,inplace=True) # drop last n rows

#Last possible run test another lapse rate
print("Finished calculating elevation stats.")


In [ ]:
df_ensemble_holder

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(20,12), dpi=300)

ens_mean = axes.plot(ens_df['ENS_MEAN'], ens_df.index, marker='o', color="red", linewidth=1.5, zorder=6, label="ENS MEAN")
ens_range = axes.fill_betweenx(ens_df.index, ens_df['ENS_MEAN']-ens_df['ENS_STD'],ens_df['ENS_MEAN']+ens_df['ENS_STD'],
                                    color="brown", alpha=0.5, label = "$\pm$ Std. deviation")
#ens_min = axes.plot(ens_df['Physical_Min'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MIN")
#ens_max = axes.plot(ens_df['Physical_Max'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MAX")
## for each column in new df add plot
for column in df_ensemble_holder.columns:
    axes.plot(df_ensemble_holder[column], df_ensemble_holder.index, alpha=0.2, linewidth=1., zorder=6, color="brown")
axes.plot(mb['ANNUAL_BALANCE'], mb.index, marker='o', color="black", linewidth=1.5, zorder=6, label="WGMS")
axes.axvline(x=0, linestyle='--', color="black")
axes.set_xlim(-8,3)
axes.set_ylim(2400,3750)
#axes[0].axhline(y=3400, color="red", linestyle="dashed")
axes.set_xticks(np.arange(-7,3+0.75,0.75))
axes.set_yticks(np.arange(2400,3750+50,100))
axes.set_yticklabels(axes.get_yticks(), fontsize=18)
axes.set_xticklabels(axes.get_xticks(), rotation=45, fontsize=18)
axes.set_xlabel("Mass Balance (m w.e. a$^{-1}$)")
ax2 = axes.twiny()
ax2.barh(ens_df.index+7.5, area_per_bin, align='center', height=15, color="red", alpha=0.6, edgecolor="white", zorder=-1)
ax2.barh(mb.index-7.5, mb['AREA'], align='center', height=15, color="black", alpha=0.6, edgecolor="white", zorder=-1)
ax2.set_xticks(np.arange(0,2.+0.5,0.5))
ax2.set_xlim(0,2.)
ax2.set_xlabel("Area (km²)")
axes.set_ylabel("Elevation [m a.s.l.]")
axes.grid()
axes.set_zorder(ax2.get_zorder()+1)
axes.set_frame_on(False)

# Axis 1 - repeat for year 2003/04
# Missing data for that.


handles, labels = axes.get_legend_handles_labels()
fig.tight_layout()
fig.legend(handles, labels, loc='upper center', ncol=5, bbox_to_anchor=(0.5, -0.01))

#figure legend below plot
#more subspacing

#plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive - uibk.ac.at/PhD/PhD/Data/Hintereisferner/Figures/"+"lrtest_hef_mb_gradients.png", bbox_inches="tight")


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,12), dpi=300, sharey=True)


axes = ax[0]
ens_mean = axes.plot(ens_df['ENS_MEDIAN'], ens_df.index, marker='o', color="red", linewidth=1.5, zorder=6, label="ENS MEDIAN")
ens_range = axes.fill_betweenx(ens_df.index, ens_df['CI_lower'],ens_df['CI_upper'],
                                    color="brown", alpha=0.5, label = "$\pm$ Std. deviation")
#ens_min = axes.plot(ens_df['Physical_Min'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MIN")
#ens_max = axes.plot(ens_df['Physical_Max'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MAX")
## for each column in new df add plot
for column in df_ensemble_holder.columns:
    axes.plot(df_ensemble_holder[column], df_ensemble_holder.index, alpha=0.2, linewidth=1., zorder=6, color="brown")
axes.plot(mb['ANNUAL_BALANCE'], mb.index, marker='o', color="black", linewidth=1.5, zorder=6, label="WGMS")
axes.axvline(x=0, linestyle='--', color="black")
axes.set_xlim(-8,3)
axes.set_ylim(2400,3750)
#axes[0].axhline(y=3400, color="red", linestyle="dashed")
axes.set_xticks(np.arange(-7,3+0.75,0.75))
axes.set_yticks(np.arange(2400,3750+50,100))
axes.set_yticklabels(axes.get_yticks(), fontsize=18)
axes.set_xticklabels(axes.get_xticks(), rotation=45, fontsize=18)
axes.set_xlabel("Mass Balance (m w.e. a$^{-1}$)")
ax2 = axes.twiny()
ax2.barh(ens_df.index+7.5, area_per_bin, align='center', height=15, color="red", alpha=0.6, edgecolor="white", zorder=-1)
ax2.barh(mb.index-7.5, mb['AREA'], align='center', height=15, color="black", alpha=0.6, edgecolor="white", zorder=-1)
ax2.set_xticks(np.arange(0,2.+0.5,0.5))
ax2.set_xlim(0,2.)
ax2.set_xlabel("Area (km²)")
axes.set_ylabel("Elevation (m a.s.l.)")
axes.grid()
axes.set_zorder(ax2.get_zorder()+1)
axes.set_frame_on(False)

# Axis 1 - repeat for year 2003/04
ax_2 = ax[1]
ens_mean = ax_2.plot(ens_df_2004['ENS_MEDIAN'], ens_df_2004.index, marker='o', color="red", linewidth=1.5, zorder=6, label="ENS MEAN")
ens_range = ax_2.fill_betweenx(ens_df_2004.index, ens_df_2004['CI_lower'], ens_df_2004['CI_upper'],
                                    color="brown", alpha=0.5, label = "$\pm$ Std. deviation")
#ens_min = ax2.plot(ens_df['Physical_Min'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MIN")
#ens_max = ax2.plot(ens_df['Physical_Max'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MAX")
## for each column in new df add plot
for column in df_ensemble_holder_2004.columns:
    ax_2.plot(df_ensemble_holder_2004[column], df_ensemble_holder_2004.index, alpha=0.2, linewidth=1., zorder=6, color="brown")
ax_2.plot(mb_2004['ANNUAL_BALANCE'], mb_2004.index, marker='o', color="black", linewidth=1.5, zorder=6, label="WGMS")
ax_2.axvline(x=0, linestyle='--', color="black")
ax_2.axhline(y=2640, linestyle='--', color="black")
ax_2.axhline(y=3048, linestyle='--', color="black")
ax_2.set_xlim(-8,3)
ax_2.set_ylim(2400,3750)
#axes[0].axhline(y=3400, color="red", linestyle="dashed")
ax_2.set_xticks(np.arange(-7,3+0.75,0.75))
ax_2.set_yticks(np.arange(2400,3750+50,100))
ax_2.set_yticklabels(ax_2.get_yticks(), fontsize=18)
ax_2.set_xticklabels(ax_2.get_xticks(), rotation=45, fontsize=18)
ax_2.set_xlabel("Mass Balance (m w.e. a$^{-1}$)")
ax2 = ax_2.twiny()
ax2.barh(ens_df_2004.index+7.5, area_per_bin, align='center', height=15, color="red", alpha=0.6, edgecolor="white", zorder=-1)
ax2.barh(mb_2004.index-7.5, mb_2004['AREA'], align='center', height=15, color="black", alpha=0.6, edgecolor="white", zorder=-1)
ax2.set_xticks(np.arange(0,2.+0.5,0.5))
ax2.set_xlim(0,2.)
ax2.set_xlabel("Area (km²)")
ax_2.set_ylabel("Elevation (m a.s.l.)")
ax_2.grid()
ax_2.set_zorder(ax2.get_zorder()+1)
ax_2.set_frame_on(False)

## calculate metrics and annotate
rmse, r2, mae = calc_metrics(mb['ANNUAL_BALANCE'], ens_df['ENS_MEDIAN'])
axes.annotate("2002 to 2009\nR²: {},\nMAE; {} m w.e.,\nRMSE: {} m w.e.".format(round(r2, 2), round(mae,2), round(rmse,2)), xy=(0.25, 0.8), xytext=(0.25, 0.8),
              textcoords='axes fraction', xycoords='axes fraction', size=22)
rmse, r2, mae = calc_metrics(mb_2004['ANNUAL_BALANCE'], ens_df_2004['ENS_MEDIAN'])
ax_2.annotate("2003/04\nR²: {},\nMAE; {} m w.e.,\nRMSE: {} m w.e.".format(round(r2, 2), round(mae,2), round(rmse,2)), xy=(0.25, 0.8), xytext=(0.25, 0.8),
              textcoords='axes fraction', xycoords='axes fraction', size=22)
ax_2.annotate("AWS Lower", xy=(0.72, 0.15), textcoords="axes fraction", xycoords="axes fraction", size=22)
ax_2.annotate("AWS Upper", xy=(0.72, 0.445), textcoords="axes fraction", xycoords="axes fraction", size=22)

handles, labels = axes.get_legend_handles_labels()
fig.tight_layout()
fig.legend(handles, labels, loc='upper center', ncol=5, bbox_to_anchor=(0.5, -0.01))

#figure legend below plot
#more subspacing
"""
if 'win' in sys.platform:
    plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/"+"hef_mb_gradients.png", bbox_inches="tight")
else:
    plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/"+"hef_mb_gradients.png", bbox_inches="tight")
"""

In [ ]:
wgms_mb_test  =  wgms_mb.loc[wgms_mb['YEAR'].isin(pd.date_range("2002-01-01", "2009-01-01", freq="1YS"))]
wgms_mb_test

In [ ]:
#manually digitized from paper
klug_etal_geod = np.array([-0.685,-2.713,-0.654,-1.028,-2.091,-1.363,-1.252,-1.209])
klug_etal_unc = np.array([0.062, 0.183, 0.063, 0.056, 0.1, 0.041, 0.046, 0.06])
wgms_mb_test['klug_mb'] = klug_etal_geod
wgms_mb_test['klug_unc'] = klug_etal_unc
wgms_mb_test

In [ ]:
## test correlation between the series
mb_df = wgms_mb_test[['ANNUAL_BALANCE','klug_mb']]
mb_df['wgms'] = mb_df['ANNUAL_BALANCE'] / 1000
mb_df['COSIPY'] = median_mb.values/1000
mb_df.drop('ANNUAL_BALANCE', axis=1, inplace=True)
corr_matrix = mb_df[['COSIPY', 'wgms', 'klug_mb']].corr(method='pearson')
print(corr_matrix)

In [ ]:
def performance_metrics(obs, sim):
    """Compute agreement metrics between observed and simulated series."""
    obs, sim = np.array(obs), np.array(sim)

    r = np.corrcoef(obs, sim)[0, 1]  # Pearson correlation
    mbe = np.mean(sim - obs)          # Mean Bias Error
    mae = np.mean(np.abs(sim - obs))  # Mean Absolute Error
    rmse = np.sqrt(np.mean((sim - obs)**2))  # Root Mean Square Error
    nse = 1 - np.sum((sim - obs)**2) / np.sum((obs - np.mean(obs))**2)  # Nash–Sutcliffe Efficiency
    slope = np.cov(obs, sim)[0, 1] / np.var(obs)  # Regression slope
    var_ratio = np.var(sim) / np.var(obs)

    return pd.Series({
        'r': r,
        'MBE': mbe,
        'MAE': mae,
        'RMSE': rmse,
        'NSE': nse,
        'slope': slope,
        'var_ratio': var_ratio
    })

# Compute metrics for all pairs
cols = ['COSIPY', 'wgms', 'klug_mb']
results = {}

for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        a, b = cols[i], cols[j]
        key = f"{a} vs {b}"
        results[key] = performance_metrics(mb_df[a], mb_df[b])

results_df = pd.DataFrame(results).T
print(results_df.round(2))

In [ ]:
median_mb/1000

In [ ]:
ci_lower_mb/1000

In [ ]:
ci_upper_mb/1000

In [ ]:
def prep_area_df(wgms_df, year):
    df = wgms_df.loc[wgms_df['YEAR'] == year]
    df = df[df['LOWER_BOUND'] != 9999]
    df['LOWER_BOUND'] = df['LOWER_BOUND'] + 25
    df.set_index('LOWER_BOUND', inplace=True)
    df.drop(['WGMS_ID','WINTER_BALANCE','WINTER_BALANCE_UNC','SUMMER_BALANCE','SUMMER_BALANCE_UNC'], axis=1, inplace=True)
    return df
    
## Prepare the areas
area_2002 = prep_area_df(wgms, 2002)
area_2004 = prep_area_df(wgms, 2004)
area_2009 = prep_area_df(wgms, 2009)
area_2002

In [ ]:
## Compare MB at elevations (mean 2002 - 2009)
ens_df['ENS_MEDIAN']

In [ ]:
mb['ANNUAL_BALANCE']

In [ ]:
ens_df['ENS_MEDIAN']-mb['ANNUAL_BALANCE']

In [ ]:
(ens_df['ENS_MEDIAN']-mb['ANNUAL_BALANCE']).iloc[4:16].mean()

In [ ]:
plt.rc('axes', axisbelow=True)
#fig, ax = plt.subplots(1,2, figsize=(20,12), dpi=300)
fig = plt.figure(figsize=(24,16), dpi=300)
gs = gridspec.GridSpec(2, 2, height_ratios=[1, 3])  # 2 rows, 2 cols

# Top panel spanning both columns
ax_top = fig.add_subplot(gs[0, :])  # row 0, all columns

# Bottom two panels
ax_left = fig.add_subplot(gs[1, 0])  # row 1, col 0
ax_right = fig.add_subplot(gs[1, 1])  # row 1, col 1

gs.update(hspace=0.35)
years_int = median_mb.index.year  # This gives an Int64Index with just the year numbers

bar_width = 0.25  # width of each bar
x = np.arange(len(years_int))  # x locations for bars

# Ensemble bars: shifted left by half bar width
ax_top.bar(x - bar_width, median_mb.values/1000, bar_width, 
           yerr=[(median_mb.values - ci_lower_mb)/1000, (ci_upper_mb - median_mb.values)/1000], 
           capsize=5, label='ENS MEDIAN', color='indianred', alpha=0.9)

# WGMS bars: shifted right by half bar width
# Align wgms years to the same x scale - assuming same years and order
ax_top.bar(x, wgms_mb_test['ANNUAL_BALANCE']/1000, bar_width,
           yerr=wgms_mb_test['ANNUAL_BALANCE_UNC']/1000, capsize=5,
           label='WGMS', color='black', alpha=0.9)

ax_top.bar(x + bar_width, wgms_mb_test['klug_mb'], bar_width,
           yerr=wgms_mb_test['klug_unc'], capsize=5,
           label='Klug et al., 2018', color='steelblue', alpha=0.9)

# Set x-ticks to middle of grouped bars and label with years
ax_top.set_xticks(x)
ax_top.set_xticklabels(years_int, rotation=30)
ax_top.set_yticks(np.arange(-3, 0.5+0.5,1))

#ax_top.set_xlabel('Year')
ax_top.set_ylabel('MB (m w.e. a$^{-1}$)')
ax_top.legend()
ax_top.grid(True)

#
axes = ax_left
ens_mean = axes.plot(ens_df['ENS_MEDIAN'], ens_df.index, marker='o', color="indianred", linewidth=1.5, zorder=6, label="ENS MEDIAN")
ens_range = axes.fill_betweenx(ens_df.index, ens_df['CI_lower'],ens_df['CI_upper'],
                                    color="rosybrown", alpha=0.5, label = "95% CI")
#ens_min = axes.plot(ens_df['Physical_Min'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MIN")
#ens_max = axes.plot(ens_df['Physical_Max'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MAX")
## for each column in new df add plot
for column in df_ensemble_holder.columns:
    axes.plot(df_ensemble_holder[column], df_ensemble_holder.index, alpha=0.1, linewidth=0.5, zorder=5, color="rosybrown")
axes.plot(mb['ANNUAL_BALANCE'], mb.index, marker='o', color="black", linewidth=1.5, zorder=6, label="WGMS")
axes.axvline(x=0, linestyle='--', color="black")
axes.set_xlim(-8,3)
axes.set_ylim(2400,3750)
#axes[0].axhline(y=3400, color="red", linestyle="dashed")
axes.set_xticks(np.arange(-7,3+0.75,0.75))
axes.set_yticks(np.arange(2400,3750+50,100))
axes.set_yticklabels(axes.get_yticks(), fontsize=18)
axes.set_xticklabels(axes.get_xticks(), rotation=45, fontsize=18)
axes.set_xlabel("MB (m w.e. a$^{-1}$)")
ax2 = axes.twiny()
ax2.barh(ens_df.index+7.5, area_per_bin, align='center', height=15, color="red", alpha=0.6, edgecolor="white", zorder=-1)
ax2.barh(area_2002.index-7.5, area_2002['AREA'], align='center', height=15, color="blue", alpha=0.5, edgecolor="white", zorder=-1)
ax2.barh(area_2009.index-7.5, area_2009['AREA'], align='center', height=15, color="black", alpha=0.5, edgecolor="white", zorder=-1)
ax2.set_xticks(np.arange(0,2.+0.5,0.5))
ax2.set_xlim(0,2.)
ax2.set_xlabel("Area (km²)")
axes.set_ylabel("Elevation (m a.s.l.)")
axes.grid(True, zorder=1)
#axes.set_zorder(ax2.get_zorder()+1)
axes.set_frame_on(False)

# Axis 1 - repeat for year 2003/04
ax_2 = ax_right
ens_mean = ax_2.plot(ens_df_2004['ENS_MEDIAN'], ens_df_2004.index, marker='o', color="indianred", linewidth=1.5, zorder=6, label="ENS MEDIAN")
ens_range = ax_2.fill_betweenx(ens_df_2004.index, ens_df_2004['CI_lower'], ens_df_2004['CI_upper'],
                                    color="rosybrown", alpha=0.5, label = "95% CI")
#ens_min = ax2.plot(ens_df['Physical_Min'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MIN")
#ens_max = ax2.plot(ens_df['Physical_Max'], ens_df.index, color="red", linewidth=1.,linestyle="dashed", zorder=6, label= "ENS MAX")
## for each column in new df add plot
for column in df_ensemble_holder_2004.columns:
    ax_2.plot(df_ensemble_holder_2004[column], df_ensemble_holder_2004.index, alpha=0.1, linewidth=0.5, zorder=5, color="rosybrown")
ax_2.plot(mb_2004['ANNUAL_BALANCE'], mb_2004.index, marker='o', color="black", linewidth=1.5, zorder=6, label="WGMS")
ax_2.axvline(x=0, linestyle='--', color="black")
ax_2.axhline(y=2640, linestyle='--', color="black")
ax_2.axhline(y=3048, linestyle='--', color="black")
ax_2.set_xlim(-8,3)
ax_2.set_ylim(2400,3750)
#axes[0].axhline(y=3400, color="red", linestyle="dashed")
ax_2.set_xticks(np.arange(-7,3+0.75,0.75))
ax_2.set_yticks(np.arange(2400,3750+50,100))
ax_2.set_yticklabels(ax_2.get_yticks(), fontsize=18)
ax_2.set_xticklabels(ax_2.get_xticks(), rotation=45, fontsize=18)
ax_2.set_xlabel("MB (m w.e. a$^{-1}$)")
ax2 = ax_2.twiny()
ax2.barh(ens_df_2004.index+7.5, area_per_bin, align='center', height=15, color="red", alpha=0.6, edgecolor="white", zorder=-1)
ax2.barh(area_2004.index-7.5, area_2004['AREA'], align='center', height=15, color="black", alpha=0.6, edgecolor="white", zorder=-1)
ax2.set_xticks(np.arange(0,2.+0.5,0.5))
ax2.set_xlim(0,2.)
ax2.set_xlabel("Area (km²)")
ax_2.set_ylabel("Elevation (m a.s.l.)")
ax_2.grid(True, zorder=1)
#ax_2.set_zorder(ax2.get_zorder()+1)
ax_2.set_frame_on(False)

#
fig.text(0.07, 0.89, 'a)', transform=fig.transFigure, fontsize=24)
fig.text(0.07, 0.63, 'b)', transform=fig.transFigure, fontsize=24)
fig.text(0.52, 0.63, 'c)', transform=fig.transFigure, fontsize=24)

## calculate metrics and annotate
rmse, r2, mae = calc_metrics(mb['ANNUAL_BALANCE'], ens_df['ENS_MEDIAN'])
axes.annotate("2002 to 2009\nR²: {},\nMAE; {} m w.e.,\nRMSE: {} m w.e.".format(round(r2, 2), round(mae,2), round(rmse,2)), xy=(0.25, 0.7), xytext=(0.25, 0.7),
              textcoords='axes fraction', xycoords='axes fraction', size=22)
rmsen, r2, maen = calc_metrics(mb_2004['ANNUAL_BALANCE'], ens_df_2004['ENS_MEDIAN'])
ax_2.annotate("2003/04\nR²: {},\nMAE; {} m w.e.,\nRMSE: {} m w.e.".format(round(r2, 2), round(maen,2), round(rmsen,2)), xy=(0.25, 0.7), xytext=(0.25, 0.7),
              textcoords='axes fraction', xycoords='axes fraction', size=22)
ax_2.annotate("Lower AWS", xy=(0.72, 0.14), textcoords="axes fraction", xycoords="axes fraction", size=22)
ax_2.annotate("Upper AWS", xy=(0.72, 0.444), textcoords="axes fraction", xycoords="axes fraction", size=22)

handles, labels = axes.get_legend_handles_labels()
fig.tight_layout()
fig.legend(handles, labels, loc='upper center', ncol=5, bbox_to_anchor=(0.5, 0.025))

if 'win' in sys.platform:
    plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/"+"fig07_hef_mb_gradients.pdf", bbox_inches="tight")
else:
    plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/"+"fig07_hef_mb_gradients.pdf", bbox_inches="tight")


In [ ]:
mb['ANNUAL_BALANCE'].mean()

In [ ]:
ens_df['ENS_MEAN'].mean()

In [ ]:
## MB time series? 


## Add all evaluation plots - MB time series (annual values)
## Ablation stakes - how? all in one plot